# Nombres
(Grupo 5)

Victor Tello Carrascal

Luis Gabriel Román Santillán

Mateo González de Miguel

###  PARTE IV:  Resolver los problemas dados.  El enunciado se os dará de forma separada.


Para resolver este problema hemos definido como estado (lista de posiciones(booleanos), linterna(bool), tiempo). Para comprobar que hemos llegado a una solución se compara el estado actual con las diferentes restricciones del problema (posición de personas(todos a la derecha), linterna(derecha), tiempo (mayor o igual a 0)). La primera llamada al constructor de la clase Puente se realiza poniendo el tiempo (en minutos) y la lista de tiempos.

In [ ]:
# Cargamos el módulo con los algoritmos de búsqueda.
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima-python/')
import search
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [ ]:
   class Problem2(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, lista, initial, tiempo):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.lista = lista
        self.initial = initial
        self.tiempo = tiempo
        self.batRestante = 0

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        esGoal = 1
        if state[2] < 0 or state[1] == 0:
          esGoal = 0
        i = 0
        while esGoal and i < len(state[0]):
          if state[0][i] == 0:
            esGoal = 0
          i = i + 1
        if esGoal and state[2] > self.batRestante:
          self.batRestante= state[2]
        return esGoal 

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        opcion = action.split()

        if opcion[0] == "Cruzar":
          pos1 = int(opcion[1])
          pos2 = int(opcion[3])

          if self.lista[pos1] > self.lista[pos2]:
            c = self.lista[pos1]
          else: 
            c = self.lista[pos2]
          return c
        else:
          pos = int(opcion[1])
          return  self.lista[pos]

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, state, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [ ]:
class Puente(Problem2):
    def __init__(self, tiempo, lista):
       #([false, false, false, false, false], false, tiempo) 

        initialAux = []
        #inicializo el array objetivo(1´s) y la lista inicial(0´s)

        for i in range(len(lista)):
            initialAux.append(0)
        
        initial = tuple(initialAux)
        t = tiempo * 60

        initial = (initial, 0, t)

        Problem2.__init__(self, lista, initial, t)

    def actions(self, state):## funcion de testeo
        '''Devuelve las acciones validas para un estado.'''

        #acciones: ir con dos personas o volver
        #cada estado tiene 3 componentes(listaDeMarcaje, linterna, tiempo)

        accs=list()
        marcaje = state[0] #lista de marcaje
        if state[1] == 1: 
            for i in range(len(self.lista)):
                if marcaje[i] == 1:
                    accs.append("Volver " + str(i)) #Volver '1'
        else:
            for i in range(len(self.lista)):
                if marcaje[i] == 0:
                    for j in range(len(self.lista)):
                        if j > i and marcaje[j] == 0: #solo cruzo con los mayores que yo porque 
                                                        #ya he cruzado con algún menor
                            accs.append("Cruzar " + str(i) + " y " + str(j))  #Cruzar '1' y '2'
        
        return accs   

    def result(self, state, accion):
        '''Devuelve el estado resultante de aplicar una accion a un estado
        determinado.'''
        l = self.lista #lista de numeros
        marcaje = list(state[0]) #lista de marcaje
        linterna = state[1] #linterna 0->IZQ, 1->DER
        tiempo = state[2] #tiempo
        opcion = accion.split()
        
        if tiempo > 0:#si el tiempo es positivo
            if opcion[0] == "Volver" and linterna == 1:#si la opcion es volver y la linterna esta en la der
                pos = int(opcion[1])
                if marcaje[pos] == 1:
                    marcaje[pos] = 0
                    linterna = 0
                    tiempo -= l[pos]

            if opcion[0] == "Cruzar" and linterna == 0:#si la opcion es cruzar y la linterna esta en la izq
                pos1 = int(opcion[1])
                pos2 = int(opcion[3])
                
                if marcaje[pos1] == 0 and marcaje[pos2] == 0:
                    marcaje[pos1] = 1
                    marcaje[pos2] = 1
                    linterna = 1
                    if l[pos1] >l[pos2]:
                        mayor = l[pos1]
                    else:
                        mayor = l[pos2]
                    tiempo -= mayor
        estado = (tuple(marcaje), linterna, tiempo)
        
        return (tuple(estado))

## 

In [ ]:
puente = Puente(5, [10,30,60,80,120])
puente.initial

((0, 0, 0, 0, 0), 0, 300)

In [ ]:
depth_first_tree_search(puente).solution() #no termina ya que no hay control de repetidos

In [ ]:
%%timeit
depth_first_graph_search(puente).solution()

100 loops, best of 5: 7.15 ms per loop


In [ ]:
%%timeit
breadth_first_tree_search(puente).solution()

10 loops, best of 5: 69.2 ms per loop


In [ ]:
%%timeit
breadth_first_graph_search(puente).solution()

10 loops, best of 5: 24.6 ms per loop


In [ ]:
%%timeit
uniform_cost_search(puente).solution()

10 loops, best of 5: 54.7 ms per loop


In [ ]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem2):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.lista = problem.lista
        self.tiempo = problem.tiempo
        self.analizados  = 0


    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion) #errata

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

In [ ]:
def resuelve_problem(tiempo, lista, algoritmo, h=None):
    pnt = Puente(tiempo,lista)
    p = Problema_con_Analizados(pnt)

    if h:
      sol= algoritmo(p, h).solution()
    else:
      sol= algoritmo(p).solution()
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h:
      print("Heurística: {0}".format(h.__name__))
    else:
      print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p.analizados))

    bateria = pnt.batRestante
    print("Batería Restante: " + str(bateria))




In [ ]:
resuelve_problem(5, [10,30,60,80,120], breadth_first_graph_search)

Solución: ['Cruzar 0 y 1', 'Volver 0', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 3 y 4', 'Volver 1', 'Cruzar 0 y 1']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 398
Batería Restante: 10


In [ ]:
resuelve_problem(5, [10,30,60,80,120], breadth_first_tree_search)

Solución: ['Cruzar 0 y 1', 'Volver 0', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 3 y 4', 'Volver 1', 'Cruzar 0 y 1']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 5392
Batería Restante: 10


In [ ]:
resuelve_problem(5, [10,30,60,80,120], depth_first_graph_search)

Solución: ['Cruzar 0 y 2', 'Volver 0', 'Cruzar 0 y 1', 'Volver 1', 'Cruzar 3 y 4', 'Volver 0', 'Cruzar 0 y 1']
Algoritmo: depth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 375
Batería Restante: 10


In [ ]:
resuelve_problem(5, [10,30,60,80,120], uniform_cost_search)

Solución: ['Cruzar 0 y 1', 'Volver 0', 'Cruzar 3 y 4', 'Volver 1', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 0 y 1']
Algoritmo: uniform_cost_search
Longitud de la solución: 7. Nodos analizados: 297
Batería Restante: 10


#Heurística

In [ ]:
#Problema 5 
def heu(node):
  state = node.state
  c = 0
  b = 1
  i = len(state[0])
  lista = [10,30,60,80,120]
  for i in range(len(state[0]),0,-1):
    if b == 0:
      c += lista[int(i)] 
      b = 1
    else:
      b = 0
  return c

In [ ]:
%%timeit
astar_search(puente, heu).solution()

10 loops, best of 5: 54.9 ms per loop


In [ ]:
resuelve_problem(5, [10,30,60,80,120], astar_search, heu)

Solución: ['Cruzar 0 y 1', 'Volver 0', 'Cruzar 3 y 4', 'Volver 1', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 0 y 1']
Algoritmo: astar_search
Heurística: heu
Batería Restante: 10


In [ ]:
%%timeit
best_first_graph_search(puente, heu).solution()

10 loops, best of 5: 37.8 ms per loop


In [ ]:
resuelve_problem(5, [10,30,60,80,120], best_first_graph_search, heu)

Solución: ['Cruzar 0 y 1', 'Volver 0', 'Cruzar 3 y 4', 'Volver 1', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 0 y 1']
Algoritmo: best_first_graph_search
Heurística: heu
Batería Restante: 10


# Conclusión de elección de algoritmo
En este problema, los algoritmos de búsqueda en anchura son efectivos pero no los más optimos. El algoritmo de búsqueda en profundidad con árbol no funciona por no tener control de repetidos, pero si miramos el de profundidad con grafo, vemos que es el algoritmo más eficiente entre todos los demás. El algoritmo de coste uniforme se asemeja en cuanto a coste al de anchura en árbol.

Si nos fijamos en los algoritmos que tienen en cuenta las heurísticas, podemos decir que no son los más optimos porque la heurística no es la más eficiente.

# ¿Qué longitud tiene la solución del problema para cruzar a 5 personas? 
La longitud de la solución del problema para cruzar a 5 personas es 7.

# ¿Se puede cruzar el puente con una linterna de 15 minutos de batería si hay 10 personas que tardan tiempos 10, 20, 30, 40, 50, 60, 70, 80, 120 y 120? ¿Cuál es la solución obtenida?

Se puede cruzar. La solución es:

In [ ]:
puente10 = Puente(15, [10,20,30,40,50,60,70,80,120,120])
puente10.initial


##en ningun caso estamos aplicando el result por lo que nunca va a pasar del estado inicial y al estar en el inicial 
##y aplicar la solucion solo recorro un arbol de 1 nodo


((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 0, 900)

In [ ]:
puente.actions(puente10.initial)

['Cruzar 0 y 1',
 'Cruzar 0 y 2',
 'Cruzar 0 y 3',
 'Cruzar 0 y 4',
 'Cruzar 1 y 2',
 'Cruzar 1 y 3',
 'Cruzar 1 y 4',
 'Cruzar 2 y 3',
 'Cruzar 2 y 4',
 'Cruzar 3 y 4']

In [ ]:
breadth_first_graph_search(puente10).solution()

In [ ]:
breadth_first_tree_search(puente10).solution()

In [ ]:
depth_first_graph_search(puente10).solution()

['Cruzar 8 y 9',
 'Volver 9',
 'Cruzar 7 y 9',
 'Volver 9',
 'Cruzar 2 y 5',
 'Volver 2',
 'Cruzar 0 y 2',
 'Volver 0',
 'Cruzar 0 y 1',
 'Volver 1',
 'Cruzar 6 y 9',
 'Volver 0',
 'Cruzar 0 y 1',
 'Volver 1',
 'Cruzar 3 y 4',
 'Volver 0',
 'Cruzar 0 y 1']

In [ ]:
resuelve_problem(15, [10,20,30,40,50,60,70,80,120,120], breadth_first_graph_search)

In [ ]:
resuelve_problem(15, [10,20,30,40,50,60,70,80,120,120], breadth_first_tree_search)

In [ ]:
resuelve_problem(15, [10,20,30,40,50,60,70,80,120,120], depth_first_graph_search)

Solución: ['Cruzar 8 y 9', 'Volver 9', 'Cruzar 7 y 9', 'Volver 9', 'Cruzar 2 y 5', 'Volver 2', 'Cruzar 0 y 2', 'Volver 0', 'Cruzar 0 y 1', 'Volver 1', 'Cruzar 6 y 9', 'Volver 0', 'Cruzar 0 y 1', 'Volver 1', 'Cruzar 3 y 4', 'Volver 0', 'Cruzar 0 y 1']
Algoritmo: depth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 22874
Batería Restante: 0


Hemos compilado solamente el de profundidad ya que tarda menos porque, al tener muchas soluciones, suele ser el más eficiente. Los otros podrían tardar demasiado.

Podemos decir que tiene varias soluciones ya que si ponemos un caso de ejemplo en donde el que tiene tiempo = 10 vuelve y va con otra pareja siempre, el tiempo total es 670 segundos, esto nos da ya una ventana de 230 segundos donde puede haber una gran variedad soluciones.

#Heurística

In [ ]:
#Problema 10
def heu(node):
  state = node.state
  c = 0
  b = 1
  i = len(state[0])
  lista = [10,20,30,40,50,60,70,80,120,120]
  for i in range(len(state[0]),0,-1):
    if b == 0:
      c += lista[int(i)] 
      b = 1
    else:
      b = 0
  return c

In [ ]:
%%timeit
astar_search(puente10, heu).solution()

In [ ]:
resuelve_problem(15, [10,20,30,40,50,60,70,80,120,120], astar_search, heu)

In [ ]:
%%timeit
best_first_graph_search(puente10, heu).solution()

In [ ]:
resuelve_problem(15, [10,20,30,40,50,60,70,80,120,120], best_first_graph_search, heu)

# ¿Puedes usar los algoritmos vistos para calcular la batería mínima que necesito para cruzar a las 10 personas?



Sí, al terminar de ejecutar el algoritmo, si restamos el tiempo total dado menos el atributo de la clase Problem2 llamado batRestante, resultaría en el tiempo de batería mínima necesaria para cruzar el puente.